In [1]:
import cv2
import numpy as np
from adbutils import adb
import scrcpy.core as scrcpy
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QPushButton
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtCore import QTimer, Qt

In [2]:
phones = ['Pixel 3 XL', 'Pixel 7 Pro']
adb.connect("127.0.0.1:5037")
client = scrcpy.Client(device=adb.device_list()[0])
client.start(threaded=True)
print(client.device_name)
phone_t = phones.index(client.device_name)

Pixel 7 Pro


In [3]:
update_timer = 1
def update_ui():
    screen = client.last_frame

    if screen is not None:
        scale_percent = 10
        width = int(screen.shape[1] * scale_percent / 100)
        height = int(screen.shape[0] * scale_percent / 100)
        dim = (width, height)
        resized_image = cv2.resize(screen, dim, interpolation=cv2.INTER_AREA)
        resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

        # clear_output(wait=True)
        # Update the screenshot display
        height, width, channel = resized_image.shape
        bytes_per_line = channel * width
        qimage = QImage(resized_image.data, width, height, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qimage)
        screenshot_label.setPixmap(pixmap)

    QTimer.singleShot(update_timer, update_ui)


app = QApplication(sys.argv)
window = QWidget()
window.setWindowTitle("Pokémon Information Display")
layout = QVBoxLayout()


screenshot_label = QLabel()
layout.addWidget(screenshot_label)



window.setLayout(layout)
window.show()

# Start the update loop
QTimer.singleShot(update_timer, update_ui)

app.exec_()

: 

: 